In [44]:
import cv2
import numpy as np


image = cv2.imread('C:\\Users\\Edish\\Documents\\Python Taskilled\\Project Mentorness\\Helmet Detection Project\\helmet_detection.jpg')
(h, w) = image.shape[:2]
target_size = 416

scale = min(target_size/w, target_size/h)
new_w = int(w * scale)
new_h = int(h * scale)

resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

delta_w = target_size - new_w
delta_h = target_size - new_h
top, bottom = delta_h // 2, delta_h - (delta_h // 2)
left, right = delta_w // 2, delta_w - (delta_w // 2)

color = [0, 0, 0]
padded_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

cv2.imshow("Padded Image", padded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
height, width = padded_image.shape[:2]
height, width

(416, 416)

In [46]:
config_path = 'C:\\Users\\Edish\\Documents\\Python Taskilled\\Project Mentorness\\Helmet Detection Project\\yolov3-helmet.cfg'
weights_path = 'C:\\Users\\Edish\\Documents\\Python Taskilled\\Project Mentorness\\Helmet Detection Project\\yolov3-helmet.weights'
names_path = 'C:\\Users\\Edish\\Documents\\Python Taskilled\\Project Mentorness\\Helmet Detection Project\\helmet.names'

with open(names_path, 'r') as f:
    classes = f.read().strip().split('\n')



In [47]:
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)


In [48]:
net.getUnconnectedOutLayers()


array([200, 227, 254])

In [49]:
blob = cv2.dnn.blobFromImage(padded_image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)


In [50]:
layer_names = net.getLayerNames()
layer_names

('conv_0',
 'bn_0',
 'leaky_1',
 'conv_1',
 'bn_1',
 'leaky_2',
 'conv_2',
 'bn_2',
 'leaky_3',
 'conv_3',
 'bn_3',
 'leaky_4',
 'shortcut_4',
 'conv_5',
 'bn_5',
 'leaky_6',
 'conv_6',
 'bn_6',
 'leaky_7',
 'conv_7',
 'bn_7',
 'leaky_8',
 'shortcut_8',
 'conv_9',
 'bn_9',
 'leaky_10',
 'conv_10',
 'bn_10',
 'leaky_11',
 'shortcut_11',
 'conv_12',
 'bn_12',
 'leaky_13',
 'conv_13',
 'bn_13',
 'leaky_14',
 'conv_14',
 'bn_14',
 'leaky_15',
 'shortcut_15',
 'conv_16',
 'bn_16',
 'leaky_17',
 'conv_17',
 'bn_17',
 'leaky_18',
 'shortcut_18',
 'conv_19',
 'bn_19',
 'leaky_20',
 'conv_20',
 'bn_20',
 'leaky_21',
 'shortcut_21',
 'conv_22',
 'bn_22',
 'leaky_23',
 'conv_23',
 'bn_23',
 'leaky_24',
 'shortcut_24',
 'conv_25',
 'bn_25',
 'leaky_26',
 'conv_26',
 'bn_26',
 'leaky_27',
 'shortcut_27',
 'conv_28',
 'bn_28',
 'leaky_29',
 'conv_29',
 'bn_29',
 'leaky_30',
 'shortcut_30',
 'conv_31',
 'bn_31',
 'leaky_32',
 'conv_32',
 'bn_32',
 'leaky_33',
 'shortcut_33',
 'conv_34',
 'bn_34',
 'l

In [51]:
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [52]:
output_layers

['yolo_82', 'yolo_94', 'yolo_106']

In [53]:
outputs = net.forward(output_layers)

In [54]:
boxes = []
confidences = []
class_ids = []


In [55]:
for output in outputs:
    for detection in output:
        scores = detection[5:]
        if np.sum(scores) > 0:  
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == 'Helmet':  
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, w, h) = box.astype("int")

                x = int(centerX - (w / 2))
                y = int(centerY - (h / 2))

                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

In [56]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)


In [57]:
for i in indices:
    (x, y, w, h) = boxes[i]
    cv2.rectangle(padded_image, (x, y), (x + w, y + h), (255, 255, 0), 2)
    text = 'Helmet'
    cv2.putText(padded_image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [58]:
cv2.imshow('Image', padded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
vid = cv2.VideoCapture(0)

while vid.isOpened():
    ret, frame = vid.read()
    if not ret:
        break  

    frame = cv2.resize(frame, (416, 416), interpolation=cv2.INTER_LINEAR)
    

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()

    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    class_ids = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            if np.sum(scores) > 0:  
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.9 and classes[class_id] == 'Helmet': 
                    box = detection[0:4] * np.array([width, height, width, height])
                    (centerX, centerY, w, h) = box.astype("int")

                    x = int(centerX - (w / 2))
                    y = int(centerY - (h / 2))

                    boxes.append([x, y, int(w), int(h)])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
    
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.7)
    if len(indices)>0:
        for i in indices:
            (x, y, w, h) = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
            text = 'wearing a helmet'
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        text = 'helmet not worn'
        cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    cv2.imshow("Helmet Detection", frame)

    if cv2.waitKey(1) & 0xFF==ord('q'):
        print('View is closed')
        break

    
    
    
vid.release()
cv2.destroyAllWindows()


Goruntu baglandi
